In [3]:
import pandas as pd
import numpy as np
import warnings
import json
import re

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [4]:
# Read File
file1 = open('pods_configuration.txt', 'r') 
Lines = file1.readlines() 

In [5]:
rows = []
cols = []
count = 0
# Strips the newline character 
for line in Lines: 
    #Read line
    data = line.strip()
    #Make the line into an array of chars
    res = [val for idx, val in enumerate(data) if val or (not val and data[idx - 1])]
    temp_string = ""
    temp_row = []
    mono_spaced = False # To allow single spaced strings
    for index, x in enumerate(res):
        # Check if the next char is blank
        if x == " ":
            # Omit all blank spaces except those who are after non empty strings
            if temp_string != "":
                if not mono_spaced:
                    mono_spaced = True
                    continue
                else:
                        if count == 0:
                            cols.append(temp_string) 
                        else:
                            temp_row.append(temp_string)
                        mono_spaced = False
                        temp_string = ""
                        continue
        else:
            #Check if only one space exists between two strings
            if mono_spaced:
                temp_string = temp_string + " " + x
                mono_spaced = False
            else:
                temp_string = temp_string + x
    if count == 0:
        cols.append(temp_string)
    else:
        temp_row.append(temp_string)
        rows.append(temp_row)
    count += 1

In [6]:
# Produce the Dataframe
dfile = pd.DataFrame(rows)
for idx, val in enumerate(dfile.columns):
    dfile = dfile.rename(columns={idx:str(cols[idx])})
cluster_nodes = {}

In [7]:
# Initialize Dictionary of Cluster Nodes
for idx, val in enumerate(dfile.loc[:,"NODE"]):
    if val in cluster_nodes:
        continue
    else:
        cluster_nodes[val] = {}

# Assign Pods to Specific Nodes
for idx, val in enumerate(dfile.loc[:,"NAME"]):
    curr_node = dfile.loc[idx,"NODE"]
    instances = dfile.loc[idx,"READY"]
    cluster_nodes[curr_node][val] = instances
cluster_nodes

{'gke-ixen-ixen-pool-0d9808bc-m2sm': {'apache-deployment-87f5cd9d6-k6mgr': '1/1',
  'comet-deployment-7d549f7d96-hpr6d': '1/1',
  'cygnusproxy-deployment-59dfbd659b-vk9dx': '1/1',
  'keyrock-deployment-77d5cff6fb-2jsxc': '1/1',
  'sthcometproxy-deployment-6c5df6588c-jf4fq': '1/1'},
 'gke-ixen-ixen-pool-0d9808bc-nq5w': {'authzforce-deployment-74dd75c5d5-ttgcx': '1/1',
  'cygnus-deployment-788654b5f7-l8gbg': '1/1',
  'mongo-5cd5475687-ttd6l': '1/1',
  'nodered-deployment-7bbc78b7f5-l6j8q': '1/1',
  'orionproxy-deployment-7f89d65fbb-c96j8': '1/1',
  'queryingsensors-6df56556f6-tr2ws': '1/1'},
 'gke-ixen-ixen-pool-0d9808bc-bwb9': {'mysql-deployment-8655c784d7-vfphw': '1/1',
  'noderedproxy-deployment-6cbfc88c5d-fwgff': '1/1',
  'orion-6d8595498f-xjqx8': '1/1',
  'queryingsensorsproxy-deployment-5db66c4566-567cd': '1/1'}}